# 회귀분석 과정

### 00. 기본 세팅: 패키지 import, csv 파일 읽기

In [ ]:
# 데이터 전처리 패키지
import numpy as np
import pandas as pd

# 통계학습 패키지
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 머신러닝 패키지
import sklearn.linear_model
from sklearn.model_selection import train_test_split

# 시각화 패키지
from matplotlib import pyplot as plt
import seaborn as sns

# 그래프를 실제로 그리기 위한 설정입니다
%matplotlib inline

# 경고 메시지를 무시합니다
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('~~~.csv')

# index 를 1로 시작하도록 수정
df.index += 1
df

### 01. 기본 확인(결측치, 각 변수의 데이터 타입, 기본적인 기술 통곗값)

In [ ]:
df.isnull().sum()

df.info()

In [ ]:
# 중간값, 결측치, 왜도, 첨도를 표시합니다.
df_stats = df.describe().T # 전치시키는 연산. 행과 열의 위치를 바꿔줘요. 

median_results = []
skew_results = []
kurtosis_results = []
null_results = []

for idx, val in enumerate(df_stats.index):
    median_results.append(df[val].median())
    skew_results.append(df[val].skew())
    kurtosis_results.append(df[val].kurtosis())
    null_results.append(df[val].isnull().sum())

df_stats['median'] = median_results
df_stats['missing'] = null_results
df_stats['skewness'] = skew_results
df_stats['kurtosis'] = kurtosis_results
df_stats

만약 종속 변수의 왜도가 치우쳐 있다면? 
-> 자연로그를 활용해 분포를 조정하자!

In [ ]:
np.log(df['price']).hist()
plt.show()

### 02. 데이터 전처리

In [ ]:
# 데이터를 타입별로 분류

def separate_dtype(DataFrame:df):
    df_obj = df.select_dtypes(include=['object'])
    df_numr = df.select_dtypes(include=['int64','float64'])
    return [df_obj, df_numr]

(df_obj, df_numr) = separate_dtype(df)

- column n자리만 추출
- type 변경
- 새로운 column명을 저장
- 새로운 변수 추가 등등

### 03. 시각화를 통해  데이터 특징 파악

- 히스토그램을 통해 각각의 독립 변수 데이터의 분포를 확인
- 종속 변수와 선형 관계가 있을 것 같은 독립 변수들을 선정하여 산점도(Scatter plot) 그래프 그리기
- 히트맵(Heatmap)을 통해 종속 변수(price)와 상관관계가 높은 독립 변수들은 무엇인지 확인
    - 독립 변수들 간에 상관관계가 높은 것들은 어떤 것인지 살펴보기
- 종속 변수와 상관관계가 높은 변수들만 추려서 확인(정렬 이용)
`df_corr.sort_values(by='종속변수', ascending=False)[['종속변수']]`
- 상관관계가 높게 나온 변수들의 데이터 분포 확인

-> 다중공선성이 있을 경우 상관관계가 가장 높은 변수로 분석 (아래는 예시)
```     
sns.jointplot(x='sqft_living', y='price', data=df, kind ='reg')   
plt.show()   
```   
-> 선형 관계가 있으면 단순선형회귀분석 ㄱㄱ

### 04. 단순선형회귀분석

OLS를 이용해 회귀분석을 할 때는 항상 상수항을 추가해야 함!
- add_constant 함수를 사용해서 상수항을 추가
```     
X = df[['sqft_living']]
y = df[['price']]

X = sm.add_constant(X, has_constant="add")
X.head()
```     

fit() 함수를 통해 선형 모델에 적합시키기
```
model = sm.OLS(y, X)
result_model = model.fit()
```

해당 결과를 result_model에 저장하고 summary() 함수를 통해서 결과를 확인
```
result_model.summary()
```

##### 주요 항목별 세부 내용

- Dep. Variable : 종속 변수, 타겟값.
- Model : 학습 모델 / OLS(Ordinary Least Squares), 잔차 제곱합(손실)을 최소로 하는 파라미터를 선택하는 방법
- Method : Least Squares / 잔차 제곱합(손실)을 최소로 하는 파라미터를 선택하는 방법
- No. Observation : 데이터셋 크기
- Df Residuals : 데이터셋 크기 - 추정된 파라미터 수를 뺀 것
- Df Model : 독립 변수의 숫자
- Covariance Type : 공분산 종류(default: nonrobust)
- R-squared : 결정계수, 모델의 설명력(0~1 사이 값), 1에 가까울수록 모델의 설명력이 높음
- Adj. R-squared : 조정된 결정계수, 조정된 모델의 설명력
- Std. error : 계수의 표준 오차
- F-statistic : F value
- Prob (F-statistic): p-value
- Log-Likelihood : 최대 로그 우도
- 아카이케 정보 기준(Akaike's Information Criterion; AIC) : 모델의 성능지표로 작을수록 좋은 모델
- Coef : 계수값
- P>|t| : p-value

실제 타겟값과 모델을 통해 추정한 값, 잔차(residual)를 확인
```
result_model.resid.plot()
plt.show()
```

### 05. 다중선형회귀분석

모든 변수에 대해 **다중공선성(multicollinearlity)** 문제를 확인해야 함

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

df_vif = pd.DataFrame()
df_vif["VIF"] = np.round([variance_inflation_factor(df_reg.values, i) for i in range(df_reg.shape[1])], 2)

df_vif["features"] = df_reg.columns
df_vif.sort_values(by='VIF', ascending=False)

변수를 선택해 회귀분석

In [ ]:
df_reg = df[['bedrooms','sqft_living','waterfront','view','sold-built_years','price']]

df_kc_reg = sm.add_constant(df_reg, has_constant='add')
feature_columns = list(df_kc_reg.columns.difference(['price']))

X = df_kc_reg[feature_columns]
y = np.log(df_kc_reg.price)

# 회귀 모형
multi_linear_model = sm.OLS(y, X)
result_model_2 = multi_linear_model.fit()
result_model_2.summary()

다중공선성 재확인

In [ ]:
df_vif = pd.DataFrame()
df_vif["VIF"] = np.round([variance_inflation_factor(df_reg.values, i) for i in range(df_reg.shape[1])], 2)

df_vif["features"] = df_reg.columns
df_vif.sort_values(by='VIF', ascending=False)

### 06. 회귀모형의 가정 진단

##### 1. 선형성 검정

In [ ]:
import matplotlib.pyplot as plt

fitted = result_model_2.fittedvalues

plt.scatter(fitted, y, alpha=0.5, label='Data')

# 대각선 추가
min_val = min(fitted.min(), y.min())
max_val = max(fitted.max(), y.max())
plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', label='y = x')

plt.xlabel('Fitted values')
plt.ylabel('Actual values')
plt.title('Fitted vs Actual (Log Price)')
plt.legend()
plt.show()

##### 2. 정규성 검정

In [ ]:
# Q-Q 도표
qqplot = sm.qqplot(result_model_2.resid, line="s")
plt.show() 

# 잔차의 분포
resid = result_model_2.resid
sns.histplot(resid, kde=True, bins=30)
plt.title("Residuals Distribution")
plt.show()

# 잔차 패턴 확인
fitted = result_model_2.predict()

resid = result_model_2.resid
pred = result_model_2.predict(X)

fig = plt.scatter(pred, resid, s=3)

plt.xlabel('Fitted values')
plt.ylabel('Residual')
plt.show()

# 샤피로 - 월크 검정
result_shapiro = stats.shapiro(result_model_2.resid)

print(f"F value : {result_shapiro[0]:.4f} / p-value : {result_shapiro[1]:.4f}")

if result_shapiro[1] < 0.05:
    print("p-value < 0.05 입니다.")

# 관측치 개수 확인
print(f"관측치는: {len(result_model_2.resid)}")

##### 3. 독립성 검정
다중공선성 확인하면 됨!

##### 4. 오차항의 자기상관 검정

In [ ]:
# ACF (Auto-Correlation Function)
sm.graphics.tsa.plot_acf(result_model_2.resid)
plt.show()

In [ ]:
from statsmodels.stats.stattools import durbin_watson

dw_stat = durbin_watson(result_model_2.resid)
print(f'Durbin-Watson stat: {dw_stat:.3f}')

Durbin-Watson이 2에 가까우면 자기상관이 없다 -> 오차항은 자기상관 되어있지 않다

##### 5. 등분산성

In [ ]:
sns.distplot(result_model_2.resid)
plt.show()

In [ ]:
result_model_2.resid.plot()
plt.show()

##### 6. 오차항의 평균 0인지

In [ ]:
print(np.mean(result_model_2.resid))